# CPU

In [4]:
import os

cluster_map = {"A100":"""#SBATCH -p kisski
#SBATCH -G A100:1                    
#SBATCH --mem=20G""",
"H100": """#SBATCH -p kisski-h100
#SBATCH -G H100:1                    
#SBATCH --mem=20G""",
"CPU": """#SBATCH --partition=jupyter:cpu
#SBATCH --cpus-per-task=16
#SBATCH --mem=40G"""}

format = """#!/bin/bash
#SBATCH --job-name={method}_HSIC_{dataset}
{cluster}
#SBATCH --time=12:00:00
#SBATCH --output=run_%x_%j.out
#SBATCH --error=run_%x_%j.err
#SBATCH --mail-type=FAIL # Email on start, end, failure
#SBATCH --mail-user=hao.huang@tib.eu # <-- Replace with your real email
echo "===== JOB STARTED ====="
echo "Hostname: $(hostname)"
echo "Date: $(date)"
echo "User: $USER"
# Load environment
module load miniforge3 
{cuda_use}module load gcc/13.2.0
{cuda_use}module load cuda/11.8
# module load gcc/13.2.0
# module load cuda/12.6.2
# Set up conda
source "$(conda info --base)/etc/profile.d/conda.sh"
conda activate /mnt/vast-kisski/projects/kisski-tib-activecl/cenv
# Diagnostics
echo "Which python: $(which python)"
python -c "import torch; print('Torch:', torch.__version__, '| CUDA:', torch.cuda.is_available())"
# Navigate to project directory
# Navigate to project directory
cd /mnt/vast-kisski/projects/kisski-tib-activecl/GCN_policy_ma 


# Run scripts
echo "--- Running: train_policy.py {dataset} ---"
python3 -u train_policy.py --dataset {dataset} --missing_p {miss_p} --gnn_fun {method}
# python3 -u train_policy.py --dataset {dataset} --missing_p 0.1 --gnn_fun {method}
# python3 -u train_policy.py --dataset {dataset} --missing_p 0.3 --gnn_fun {method}


echo "===== JOB COMPLETED ====="
echo "Date: $(date)"
"""

datasets_org = ['Syn_M=None_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4', 'BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1']      # network relationships not based on similarity
datasets = [e+"_MCAR" for e in datasets_org]

methods = ["GraphSAGE",'GCN']
missing_ps = [0.0,0.1,0.3]

################################################################################################################################################
cluster = 'H100'        # H100, CPU
slurm_dir = f'{cluster}_slurm'
cuda_use = "# " if cluster == 'CPU' else ""
################################################################################################################################################

os.makedirs(slurm_dir, exist_ok=True)

print("cd /mnt/vast-kisski/projects/kisski-tib-activecl/GCN_policy_ma")

for dataset in datasets:
    for method in methods:
        for miss_p in missing_ps:

            slurm_content = format.format(cluster=cluster_map[cluster], dataset=dataset, cuda_use=cuda_use,
                                          method=method, miss_p=miss_p)

            filename = os.path.join(slurm_dir, f"{method}_{dataset}_{miss_p}.slurm")

            with open(filename, "w") as f:
                f.write(slurm_content)

            sbatch_command = f"sbatch {filename}"
            print(sbatch_command)


cd /mnt/vast-kisski/projects/kisski-tib-activecl/GCN_policy_ma
sbatch H100_slurm/GraphSAGE_Syn_M=None_SimRel=1_Rel=4_MCAR_0.0.slurm
sbatch H100_slurm/GraphSAGE_Syn_M=None_SimRel=1_Rel=4_MCAR_0.1.slurm
sbatch H100_slurm/GraphSAGE_Syn_M=None_SimRel=1_Rel=4_MCAR_0.3.slurm
sbatch H100_slurm/GCN_Syn_M=None_SimRel=1_Rel=4_MCAR_0.0.slurm
sbatch H100_slurm/GCN_Syn_M=None_SimRel=1_Rel=4_MCAR_0.1.slurm
sbatch H100_slurm/GCN_Syn_M=None_SimRel=1_Rel=4_MCAR_0.3.slurm
sbatch H100_slurm/GraphSAGE_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.0.slurm
sbatch H100_slurm/GraphSAGE_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.1.slurm
sbatch H100_slurm/GraphSAGE_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.3.slurm
sbatch H100_slurm/GCN_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.0.slurm
sbatch H100_slurm/GCN_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.1.slurm
sbatch H100_slurm/GCN_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.3.slurm
sbatch H100_slurm/GraphSAGE_BlogCatalog1_M=20_SimRel=0_Rel=1_MCAR_0.0.slurm
sbatch H100_slurm/GraphSAGE_BlogCatalog1_M=20_SimRel=0_Re